In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

# Load California Housing dataset
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['PRICE'] = housing.target

print("California Housing Dataset:")
print(f"Dataset shape: {df.shape}")
print("\nFeatures description:")
for i, feature in enumerate(housing.feature_names):
    print(f"{feature}: {housing.feature_names[i]}")
print("\nTarget variable: PRICE (median house value in $100,000s)")

In [ ]:
print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nSummary statistics:")
print(df.describe())

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of California Housing Features')
plt.tight_layout()
plt.show()

In [ ]:
# Split data into features and target
X = df.drop('PRICE', axis=1)
y = df['PRICE']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test);

In [ ]:
def evaluate_model(X_train, X_test, y_train, y_test, feature_names, model_name="Model"):
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Training predictions and evaluation
    y_train_pred = model.predict(X_train)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_rmse = np.sqrt(train_mse)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Test predictions and evaluation
    y_test_pred = model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_rmse = np.sqrt(test_mse)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Cross-validation for more robust evaluation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())
    
    # Calculate adjusted R-squared for test set
    n = X_test.shape[0]  # number of observations
    p = X_test.shape[1]  # number of predictors
    test_adj_r2 = 1 - (1 - test_r2) * (n - 1) / (n - p - 1)
    
    print(f"\n--- {model_name} Evaluation ---")
    print(f"Features used: {', '.join(feature_names)}")
    print(f"Number of features: {len(feature_names)}")
    print(f"Training RMSE: {train_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"Cross-validation RMSE: {cv_rmse:.4f}")
    print(f"Training R²: {train_r2:.4f}")
    print(f"Test R²: {test_r2:.4f}")
    print(f"Test Adjusted R²: {test_adj_r2:.4f}")
    
    # Return the coefficient values along with evaluation metrics
    coefficients = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': model.coef_
    }).sort_values(by='Coefficient', ascending=False)
    
    result = {
        'model': model,
        'feature_names': feature_names,
        'test_rmse': test_rmse,
        'test_r2': test_r2,
        'test_adj_r2': test_adj_r2,
        'coefficients': coefficients,
        'cv_rmse': cv_rmse
    }
    
    return result

# Evaluate base model with all features
base_result = evaluate_model(
    X_train_scaled, X_test_scaled, y_train, y_test, 
    X.columns, "Base Model (All Features)"
);